# Ingest CXG spatial h5ad file and create SOMA object on disk

In [1]:
import tiledbsoma
from tiledbsoma.experimental.ingest import from_cxg_spatial_h5ad

In [2]:
from pathlib import Path

In [3]:
REPO_ROOT_DIR = Path("../../..")
BUILDER_DIR = REPO_ROOT_DIR / "tools/cellxgene_census_builder"
WORKING_DIR = REPO_ROOT_DIR / "issues/census_1127_spatial-builder"
ANNDATA_DIR = WORKING_DIR / "source_h5ad"
SOMA_DIR = SOMA_DIR = Path("/mnt/scratch/spatial_soma")

MANIFEST_FILE_PATH = BUILDER_DIR / "spatial_dev_tools/manifest.csv"
BLOCKLIST_FILE_PATH = BUILDER_DIR / "spatial_dev_tools/blocklist.txt"

In [4]:
SOMA_DIR.mkdir(exist_ok=True)

In [5]:
!pwd

/home/ubuntu/github/cellxgene-census/tools/cellxgene_census_builder/spatial_dev_tools


In [6]:
list(ANNDATA_DIR.glob("*.h5ad"))

[PosixPath('../../../issues/census_1127_spatial-builder/source_h5ad/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec.h5ad'),
 PosixPath('../../../issues/census_1127_spatial-builder/source_h5ad/9e66d65a-385e-4097-9e61-2ad68a1a59fb.h5ad'),
 PosixPath('../../../issues/census_1127_spatial-builder/source_h5ad/f6fbb333-bf7a-4402-a3c0-ef2c894d4371.h5ad'),
 PosixPath('../../../issues/census_1127_spatial-builder/source_h5ad/6811c454-def2-4d9e-b360-aa8a69f843ce.h5ad'),
 PosixPath('../../../issues/census_1127_spatial-builder/source_h5ad/1fee1684-94b3-4371-8d44-6a8b937ba23f.h5ad'),
 PosixPath('../../../issues/census_1127_spatial-builder/source_h5ad/5487249a-b034-446a-8d9b-0b810ecb7e91.h5ad')]

In [7]:
# NOTE: Replace value of `my_h5ad_path` with an appropriate path to a cellxgene h5ad spatial file
dataset_id = "17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec"
my_h5ad_path = ANNDATA_DIR / f"{dataset_id}.h5ad"
my_soma_path = SOMA_DIR / dataset_id

In [8]:
if my_soma_path.is_dir():
    !rm -r $my_soma_path

In [9]:
from_cxg_spatial_h5ad(
    input_h5ad_path=my_h5ad_path,
    experiment_uri=str(my_soma_path),
    measurement_name="RNA",
    scene_name=dataset_id,
    uns_keys=[],
)

<frozen abc>:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.



'/mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec'

## Inspect the structure of the SOMA object on disk

In [10]:
sp = tiledbsoma.open(str(my_soma_path))
sp

<Experiment 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec' (open for 'r') (4 items)
    'ms': 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/ms' (unopened)
    'obs': 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/obs' (unopened)
    'obs_scene': 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/obs_scene' (unopened)
    'spatial': 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/spatial' (unopened)>

In [11]:
sp.ms

<Collection 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/ms' (open for 'r') (2 items)
    'RNA': 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/ms/RNA' (unopened)
    'raw': 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/ms/raw' (unopened)>

In [12]:
sp.ms["RNA"]

<Measurement 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/ms/RNA' (open for 'r') (5 items)
    'X': 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/ms/RNA/X' (unopened)
    'obsm': 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/ms/RNA/obsm' (unopened)
    'uns': 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/ms/RNA/uns' (unopened)
    'var': 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/ms/RNA/var' (unopened)
    'var_scene': 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/ms/RNA/var_scene' (unopened)>

In [13]:
sp.ms["RNA"].var_scene

<DataFrame 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/ms/RNA/var_scene' (open for 'r')>

In [14]:
var_scene_df = sp.ms["RNA"].var_scene
var_scene_df.read().concat().to_pandas()

,soma_joinid,scene_id,data
0,6,17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec,True
1,14,17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec,True
2,16,17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec,True
3,17,17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec,True
4,18,17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec,True
...,...,...,...
15582,36388,17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec,True
15583,36389,17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec,True
15584,36392,17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec,True
15585,36404,17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec,True


In [15]:
sp.obs_scene

<DataFrame 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/obs_scene' (open for 'r')>

In [16]:
obs_scene_df = sp.obs_scene
obs_scene_df.read().concat().to_pandas()

,soma_joinid,scene_id,data
0,1,17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec,True
1,2,17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec,True
2,5,17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec,True
3,6,17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec,True
4,9,17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec,True
...,...,...,...
3368,4984,17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec,True
3369,4986,17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec,True
3370,4987,17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec,True
3371,4989,17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec,True


In [17]:
sp.spatial

<Collection 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/spatial' (open for 'r') (1 item)
    '17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec': 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/spatial/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec' (unopened)>

In [18]:
sp.spatial[dataset_id]

<Scene 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/spatial/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec' (open for 'r') (3 items)
    'img': 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/spatial/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/img' (unopened)
    'obsl': 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/spatial/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/obsl' (unopened)
    'varl': 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/spatial/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/varl' (unopened)>

In [19]:
sp.spatial[dataset_id]["img"]["hires"]

<DenseNDArray 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/spatial/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/img/hires' (open for 'r')>

In [20]:
hires_ndarray = sp.spatial[dataset_id]["img"]["hires"].read()
hires_ndarray.to_numpy()

array([[[142, 142, 141, ..., 139, 139, 139],
        [143, 142, 141, ..., 139, 140, 139],
        [141, 141, 140, ..., 140, 140, 140],
        ...,
        [141, 141, 142, ..., 140, 139, 138],
        [139, 140, 142, ..., 138, 139, 139],
        [139, 140, 140, ..., 140, 139, 139]],

       [[143, 143, 143, ..., 142, 141, 141],
        [143, 143, 143, ..., 142, 141, 141],
        [143, 143, 143, ..., 141, 142, 142],
        ...,
        [143, 144, 144, ..., 144, 144, 144],
        [143, 144, 144, ..., 144, 144, 144],
        [143, 144, 144, ..., 144, 144, 144]],

       [[145, 146, 145, ..., 144, 144, 143],
        [145, 145, 144, ..., 144, 144, 144],
        [146, 145, 145, ..., 144, 144, 143],
        ...,
        [145, 145, 145, ..., 143, 144, 144],
        [145, 145, 145, ..., 144, 144, 144],
        [145, 145, 145, ..., 145, 144, 144]]], dtype=uint8)

In [21]:
sp.spatial[dataset_id]["obsl"]

<Collection 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/spatial/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/obsl' (open for 'r') (1 item)
    'loc': 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/spatial/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/obsl/loc' (unopened)>

In [22]:
sp.spatial[dataset_id]["obsl"]["loc"]

<DataFrame 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/spatial/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/obsl/loc' (open for 'r')>

In [23]:
obsl_loc_df = sp.spatial[dataset_id]["obsl"]["loc"]
obsl_loc = obsl_loc_df.read().concat().to_pandas()
obsl_loc

,y,x,soma_joinid,in_tissue,array_row,array_col,n_counts,n_genes,percent.mt,Adipocyte,...,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage,observation_joinid,_soma_geometry
0,1275,803,475,0,0,0,NaN,NaN,NaN,NaN,...,unknown,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,RtI>4TTINp,28.788702
1,1275,891,1152,0,0,2,NaN,NaN,NaN,NaN,...,unknown,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,NK3qC{8s2W,28.788702
2,1276,1334,1619,0,0,12,NaN,NaN,NaN,NaN,...,unknown,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,Wni<XFHdla,28.788702
3,1276,1511,0,0,0,16,NaN,NaN,NaN,NaN,...,unknown,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,S{qkR!yc~8,28.788702
4,1276,1600,1569,0,0,18,NaN,NaN,NaN,NaN,...,unknown,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,qhIivxsH@t,28.788702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4987,6673,6202,4256,1,70,122,8137.0,3234.0,42.409532,0.003529,...,cardiac muscle myoblast,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,y6Rl8v6A2C,28.788702
4988,6751,6247,4672,0,71,123,NaN,NaN,NaN,NaN,...,unknown,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,xdH~?B>hRc,28.788702
4989,6751,6424,4678,0,71,127,NaN,NaN,NaN,NaN,...,unknown,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,v+}g+r2P+{,28.788702
4990,7059,6247,4697,0,75,123,NaN,NaN,NaN,NaN,...,unknown,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,lxu-C=ME;c,28.788702


In [24]:
sp.obs.read().concat().to_pandas()

,soma_joinid,obs_id,in_tissue,array_row,array_col,n_counts,n_genes,percent.mt,Adipocyte,Cardiomyocyte,...,tissue_type,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage,observation_joinid
0,0,AAACAACGAATAGTTC-1,0,0,16,NaN,NaN,NaN,NaN,NaN,...,tissue,unknown,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,S{qkR!yc~8
1,1,AAACAAGTATCTCCCA-1,1,50,102,6343.0,2678.0,50.154429,0.000089,0.658615,...,tissue,cardiac muscle myoblast,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,gFD!+Zqd(i
2,2,AAACAATCTACTAGCA-1,1,3,43,4510.0,2258.0,44.038548,0.000047,0.496638,...,tissue,cardiac muscle myoblast,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,OsSlEREYZW
3,3,AAACACCAATAACTGC-1,0,59,19,NaN,NaN,NaN,NaN,NaN,...,tissue,unknown,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,zuU(#yRj2=
4,4,AAACAGAGCGACTCCT-1,0,14,94,NaN,NaN,NaN,NaN,NaN,...,tissue,unknown,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,-$<USDZJGm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4987,4987,TTGTTTCACATCCAGG-1,1,58,42,4240.0,2028.0,50.821334,0.000030,0.694713,...,tissue,cardiac muscle myoblast,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,!-K_Z3vDq*
4988,4988,TTGTTTCATTAGTCTA-1,0,60,30,NaN,NaN,NaN,NaN,NaN,...,tissue,unknown,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,&q#Fy^DD*S
4989,4989,TTGTTTCCATACAACT-1,1,45,27,4618.0,2040.0,47.920398,0.000010,0.714691,...,tissue,cardiac muscle myoblast,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,ZuyRYj_t<2
4990,4990,TTGTTTGTATTACACG-1,0,73,41,NaN,NaN,NaN,NaN,NaN,...,tissue,unknown,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,;D9RPjix7Z


In [25]:
obsl_loc.apply(lambda x: len(x.unique())).sort_values().head(20)

assay_ontology_term_id                      1
self_reported_ethnicity_ontology_term_id    1
is_primary_data                             1
organism_ontology_term_id                   1
sex_ontology_term_id                        1
tissue_ontology_term_id                     1
donor_id                                    1
suspension_type                             1
tissue_type                                 1
assay                                       1
disease                                     1
organism                                    1
sex                                         1
tissue                                      1
self_reported_ethnicity                     1
development_stage                           1
disease_ontology_term_id                    1
development_stage_ontology_term_id          1
_soma_geometry                              1
in_tissue                                   2
dtype: int64

In [26]:
sp.obs.read().concat().to_pandas()

,soma_joinid,obs_id,in_tissue,array_row,array_col,n_counts,n_genes,percent.mt,Adipocyte,Cardiomyocyte,...,tissue_type,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage,observation_joinid
0,0,AAACAACGAATAGTTC-1,0,0,16,NaN,NaN,NaN,NaN,NaN,...,tissue,unknown,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,S{qkR!yc~8
1,1,AAACAAGTATCTCCCA-1,1,50,102,6343.0,2678.0,50.154429,0.000089,0.658615,...,tissue,cardiac muscle myoblast,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,gFD!+Zqd(i
2,2,AAACAATCTACTAGCA-1,1,3,43,4510.0,2258.0,44.038548,0.000047,0.496638,...,tissue,cardiac muscle myoblast,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,OsSlEREYZW
3,3,AAACACCAATAACTGC-1,0,59,19,NaN,NaN,NaN,NaN,NaN,...,tissue,unknown,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,zuU(#yRj2=
4,4,AAACAGAGCGACTCCT-1,0,14,94,NaN,NaN,NaN,NaN,NaN,...,tissue,unknown,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,-$<USDZJGm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4987,4987,TTGTTTCACATCCAGG-1,1,58,42,4240.0,2028.0,50.821334,0.000030,0.694713,...,tissue,cardiac muscle myoblast,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,!-K_Z3vDq*
4988,4988,TTGTTTCATTAGTCTA-1,0,60,30,NaN,NaN,NaN,NaN,NaN,...,tissue,unknown,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,&q#Fy^DD*S
4989,4989,TTGTTTCCATACAACT-1,1,45,27,4618.0,2040.0,47.920398,0.000010,0.714691,...,tissue,cardiac muscle myoblast,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,ZuyRYj_t<2
4990,4990,TTGTTTGTATTACACG-1,0,73,41,NaN,NaN,NaN,NaN,NaN,...,tissue,unknown,Visium Spatial Gene Expression,myocardial infarction,Homo sapiens,male,heart left ventricle,European,44-year-old human stage,;D9RPjix7Z


In [27]:
sp.spatial[dataset_id]["obsl"]["loc"]

<DataFrame 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/spatial/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/obsl/loc' (open for 'r')>

In [28]:
sp.spatial[dataset_id]

<Scene 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/spatial/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec' (open for 'r') (3 items)
    'img': Collection 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/spatial/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/img' (open for 'r') (2 items)
        'fullres': 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/spatial/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/img/fullres' (unopened)
        'hires': DenseNDArray 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/spatial/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/img/hires' (open for 'r')
    'obsl': Collection 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/spatial/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/obsl' (open for 'r') (1 item)
        'loc': DataFrame 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/spatial/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/obsl/loc' (open for 'r')
    'varl': 'file

In [29]:
sp

<Experiment 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec' (open for 'r') (4 items)
    'ms': Collection 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/ms' (open for 'r') (2 items)
        'RNA': Measurement 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/ms/RNA' (open for 'r') (5 items)
            'X': 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/ms/RNA/X' (unopened)
            'obsm': 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/ms/RNA/obsm' (unopened)
            'uns': 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/ms/RNA/uns' (unopened)
            'var': 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/ms/RNA/var' (unopened)
            'var_scene': DataFrame 'file:///mnt/scratch/spatial_soma/17d9e43f-1251-4f1e-8a5b-a96f2c89e5ec/ms/RNA/var_scene' (open for 'r')
        'raw': 'file:///mnt/scratch/spatial_soma/1